In [1]:
import pandas as pd
import numpy as np

In [2]:
data1 = pd.read_excel('./data/附件1.xlsx')
data2 = pd.read_csv('./data/附件2.csv')
data3 = pd.read_excel('./data/附件3.xlsx')

In [3]:
train = data2[data2['FLAG'].isnull() == False] # 只要里面的训练集
test  = data2[data2['FLAG'].isnull() == True] # 只要里面的测试集

In [4]:
train

,TICKER_SYMBOL,ACT_PUBTIME,PUBLISH_DATE,END_DATE_REP,END_DATE,REPORT_TYPE,FISCAL_PERIOD,MERGED_FLAG,ACCOUTING_STANDARDS,CURRENCY_CD,...,CA_TURNOVER,OPER_CYCLE,INVEN_TURNOVER,FA_TURNOVER,TFA_TURNOVER,DAYS_AP,DAYS_INVEN,TA_TURNOVER,AR_TURNOVER,FLAG
0,4019,3,3,2,1,A,12,1,CHAS_2007,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,8166,3,3,2,1,A,12,1,CHAS_2007,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,9063,3,3,2,1,A,12,1,CHAS_2007,CNY,...,1.3751,148.8938,7.4240,20.9362,15.2980,75.4337,48.4911,1.2774,3.5856,0.0
3,10083,4,4,4,1,A,12,1,CHAS_2007,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,11737,3,3,2,1,A,12,1,CHAS_2007,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18055,4993201,7,7,6,5,A,12,1,CHAS_2007,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
18056,4993297,7,7,6,5,A,12,1,CHAS_2007,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
18057,4997833,7,7,6,5,A,12,1,CHAS_2007,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
18058,4998808,7,7,6,5,A,12,1,CHAS_2007,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


1）训练集正负样本不均横
* 正反样比例0:1 = 17880:180

In [5]:
train['FLAG'].value_counts()

0.0    17880
1.0      180
Name: FLAG, dtype: int64

2）训练集4152只股票、测试集4153股票
* 测试集中的3864415，是训练集中没有的

In [6]:
print(len(set(train['TICKER_SYMBOL'])))
print(len(set(test['TICKER_SYMBOL'])))

4152
4153


3）列下类别总数目为1的特征（无效）
* REPORT_TYPE（object）
* FISCAL_PERIOD
* MERGED_FLAG
* ACCOUTING_STANDARDS（object）
* CURRENCY_CD（object）

In [19]:
# 对缺失值填-1，查看方差为1的列
df = train.fillna(-1)
for i in df.columns:
    if df[i].nunique() == 1:
        print(i)

REPORT_TYPE
FISCAL_PERIOD
MERGED_FLAG
ACCOUTING_STANDARDS
CURRENCY_CD


4）数据的缺失值处理
* 查看各列缺失值的比例如下

In [10]:
df = data2[data2['FLAG'].isnull() == False].fillna(-1) #只要里面的训练集

In [20]:
## 这里不用运行
# 查看每一列的缺失比例
pd.set_option('display.max_rows',None)
pd.DataFrame(train.isnull().sum()/18060,columns=['naRate']).sort_values('naRate',ascending=False)

,naRate
ACCRUED_EXP,0.999779
PERPETUAL_BOND_L,0.997453
PREFERRED_STOCK_L,0.997287
PREFERRED_STOCK_E,0.994684
N_COMPENS_PAYOUT,0.993245
N_REINSUR_PREM,0.993079
POLICY_DIV_PAYT,0.992359
N_INC_PH_DEPOS_INV,0.992193
REINSUR_EXP,0.992193
PREM_REFUND,0.992137


5）有些股票不一定有5条数据

In [7]:
# 训练集中的股票代码不存在缺失
# 注意每个公司给的数据条数不一样，这里可能对结果会有重要影响
train['TICKER_SYMBOL'].value_counts().value_counts()

5    3094
4     367
1     362
2     227
3     102
Name: TICKER_SYMBOL, dtype: int64

6） 有些公司重复造假

In [8]:
# 可以看到有些公司会重复造假
s1 = pd.DataFrame(train.groupby(['TICKER_SYMBOL'])['FLAG'].sum())
s1[s1['FLAG'] > 1].sort_values('FLAG',ascending=False)

,FLAG
TICKER_SYMBOL,
4710457,3.0
4391931,3.0
79573,2.0
166884,2.0
4156459,2.0
3646698,2.0
3360110,2.0
3261456,2.0
3018406,2.0
